In [1]:
pip install tensorflow_datasets --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 90.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 251.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 249.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.1/156.1 kB 246.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 234.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 252.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 287.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 276.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 289.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 279.9 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21483 sha256=4fbf59b0377069296148da6a74afd11d10b469e40ddf9a4d11a14a599b

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import layers, models, optimizers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
# Now that we know the image dimensions
train_data_dir = 'data/project3/data_all_modified/data_split/train'
# Number of images we want to process at once
batch_size = 64

# Target image size (128 px by 128 px)
img_height = 128
img_width = 128
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.


In [7]:
test_data_dir = 'data/project3/data_all_modified/data_split/test'

batch_size = 2

img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 4265 files belonging to 2 classes.


In [8]:
from keras import layers
from keras import models
import pandas as pd

model_mod_lenet5 = models.Sequential()
# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 2: Convolutional layer with 32 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 3: Convolutional layer with 64 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 4: Convolutional layer with 128 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 5: Convolutional layer with 128 filters of size 3x3, followed by Max pooling
model_mod_lenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_mod_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# flattening the output of the conv layer after max pooling 
model_mod_lenet5.add(layers.Flatten())
#Dropout Layer
model_mod_lenet5.add(layers.Dropout(0.2))
# Layer 8: Fully connected layer with 120 neurons
model_mod_lenet5.add(layers.Dense(512, activation='relu'))

model_mod_lenet5.add(layers.Dense(2, activation='sigmoid'))

model_mod_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

# Generating the summary of the model
model_mod_lenet5.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 6)       168       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 6)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        1760      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 64)        0

In [79]:
# Compile model
# RMSprop (Root Mean Square Propagation) is commonly used in training deep neural networks.
model_cnn.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# Generating the summary of the model
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 64, 64, 32)        18464     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 16, 16, 32)       

In [9]:
import tensorflow as tf
model = tf.keras.models.load_model('Mod_LeNet5.keras')

In [69]:
import requests

In [3]:

iterator = iter(test_rescale_ds)
l=iterator.get_next()[0].numpy().to.list()

# make the POST request passing the single test case as the `image` field:
rsp = requests.post("http://172.17.0.1:5000/proj_models/ann/v1", json={"image":l})

NameError: name 'test_rescale_ds' is not defined

In [67]:
rsp.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Response Content: <!doctype html>
<html lang=en>
  <head>
    <title>NameError: name &#39;request&#39; is not defined
 // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script>
      var CONSOLE_MODE = false,
          EVALEX = true,
          EVALEX_TRUSTED = false,
          SECRET = "1CVEZGNzRY9wPqfB3xQQ";
    </script>
  </head>
  <body style="background-color: #fff">
    <div class="debugger">
<h1>NameError</h1>
<div class="detail">
  <p class="errormsg">NameError: name &#39;request&#39; is not defined
</p>
</div>
<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>
<div class="traceback">
  <h3></h3>
  <ul><li><div class="frame" id="frame-139942456201504">
  <h4>File <cite class="filename">"/usr/local/lib/python3.11/s